In [143]:
import numpy as np
import pandas as pd

from treen import load_example
import enflow as ef
import gymnasium as gym

### Step 1) Define the energy system 

In [144]:
df = load_example('gefcom2014-wind')

windfarms = []
for name in df.columns.levels[0]:
    windfarms.append(ef.WindFarm(capacity=1, name=name))

portfolio = ef.Portfolio(assets=windfarms)

In [145]:
df.head()

Site1                      \
                                            Power       U10      U100   
ref_datetime        valid_datetime                                      
2012-01-01 01:00:00 2012-01-01 01:00:00  0.000000  2.124600  2.864280   
                    2012-01-01 02:00:00  0.054879  2.521695  3.344859   
                    2012-01-01 03:00:00  0.110234  2.672210  3.508448   
                    2012-01-01 04:00:00  0.165116  2.457504  3.215233   
                    2012-01-01 05:00:00  0.156940  2.245898  2.957678   

                                                                Site2  \
                                              V10      V100     Power   
ref_datetime        valid_datetime                                      
2012-01-01 01:00:00 2012-01-01 01:00:00 -2.681966 -3.666076  0.596273   
                    2012-01-01 02:00:00 -1.796960 -2.464761  0.411180   
                    2012-01-01 03:00:00 -0.822516 -1.214093  0.167243   
                    2012-01-01 04:00:00 -0.143642 -0.355546  0.037326   
                    2012-01-01 05:00:00  0.389576  0.332701  0.051206   

                                                                       \
                                              U10      U100       V10   
ref_datetime        valid_datetime                                      
2012-01-01 01:00:00 2012-01-01 01:00:00 -0.171642 -0.112594 -5.466031   
                    2012-01-01 02:00:00 -0.088859 -0.014100 -4.643205   
                    2012-01-01 03:00:00 -0.246014 -0.225238 -3.613395   
                    2012-01-01 04:00:00 -0.680738 -0.772046 -2.919615   
                    2012-01-01 05:00:00 -1.261106 -1.487537 -2.619786   

                                                   ... Site9            \
                                             V100  ... Power       U10   
ref_datetime        valid_datetime                 ...                   
2012-01-01 01:00:00 2012-01-01 01:00:00 -7.101347  ...   0.0  1.903254   
                    2012-01-01 02:00:00 -5.896778  ...   0.0  2.173296   
                    2012-01-01 03:00:00 -4.489369  ...   0.0  2.312744   
                    2012-01-01 04:00:00 -3.598824  ...   0.0  2.217565   
                    2012-01-01 05:00:00 -3.244667  ...   0.0  1.900598   

                                                                       \
                                             U100       V10      V100   
ref_datetime        valid_datetime                                      
2012-01-01 01:00:00 2012-01-01 01:00:00  2.516723 -1.495002 -2.078063   
                    2012-01-01 02:00:00  2.837109 -0.643641 -1.006914   
                    2012-01-01 03:00:00  3.034096  0.136595 -0.015490   
                    2012-01-01 04:00:00  2.941829  0.797156  0.823531   
                    2012-01-01 05:00:00  2.559177  1.285656  1.471102   

                                           Site10                      \
                                            Power       U10      U100   
ref_datetime        valid_datetime                                      
2012-01-01 01:00:00 2012-01-01 01:00:00  0.594361  1.655790  2.024257   
                    2012-01-01 02:00:00  0.569679  2.178609  2.623141   
                    2012-01-01 03:00:00  0.330539  2.228633  2.645217   
                    2012-01-01 04:00:00  0.211308  2.077232  2.432947   
                    2012-01-01 05:00:00  0.172140  1.814716  2.097844   

                                                             
                                              V10      V100  
ref_datetime        valid_datetime                           
2012-01-01 01:00:00 2012-01-01 01:00:00 -4.649593 -5.992232  
                    2012-01-01 02:00:00 -4.184944 -5.318086  
                    2012-01-01 03:00:00 -3.242442 -4.052232  
                    2012-01-01 04:00:00 -2.345085 -2.913831  
                    2012-01-01 05:00:00 -1.577625 -1.976100  

[5 rows x 50 columns]

In [146]:
dataset = ef.Dataset(name="gefcom2024",
                     description="Data provided by the organisers of HEFTCom2024. Participants are free to use additional external data.",
                     energy_system=portfolio,
                     data={"data_gefcom2014_wind": df})

### Step 2) Define state, exogeneous and action spaces

In [147]:
state_space = ef.DataFrameSpace({asset.name: {
    'Power': gym.spaces.Box(low=0, high=1, shape=(1,), dtype=np.float32)
} for asset in portfolio.assets})

exogeneous_space = ef.DataFrameSpace({asset.name: {
    'U10': gym.spaces.Box(low=-np.inf, high=np.inf, shape=(1,), dtype=np.float32),
    'V10': gym.spaces.Box(low=-np.inf, high=np.inf, shape=(1,), dtype=np.float32),
    'U100': gym.spaces.Box(low=-np.inf, high=np.inf, shape=(1,), dtype=np.float32),
    'V100': gym.spaces.Box(low=-np.inf, high=np.inf, shape=(1,), dtype=np.float32)
} for asset in portfolio.assets})

action_space = ef.DataFrameSpace({asset.name: {
    f"Quantile_forecast_{i+1}": gym.spaces.Box(low=0, high=1, shape=(1,)) for i in range(99)
} for asset in portfolio.assets})


In [148]:
action_space.sample(n_rows=3)

Site1                                            \
  Quantile_forecast_1 Quantile_forecast_10 Quantile_forecast_11   
0            0.837348             0.360967             0.894261   
1            0.682897             0.408320             0.821909   
2            0.348161             0.748320             0.047880   

                                                                  \
  Quantile_forecast_12 Quantile_forecast_13 Quantile_forecast_14   
0             0.045181             0.924268             0.207248   
1             0.505007             0.434750             0.653865   
2             0.846367             0.670771             0.775054   

                                                                  \
  Quantile_forecast_15 Quantile_forecast_16 Quantile_forecast_17   
0             0.039155             0.592308             0.693280   
1             0.947773             0.924092             0.576941   
2             0.563875             0.066391             0.871596   

                        ...                Site9                       \
  Quantile_forecast_18  ... Quantile_forecast_90 Quantile_forecast_91   
0             0.067031  ...             0.579525             0.407737   
1             0.096989  ...             0.004608             0.561656   
2             0.147020  ...             0.875101             0.937921   

                                                                  \
  Quantile_forecast_92 Quantile_forecast_93 Quantile_forecast_94   
0             0.636381             0.997480             0.911098   
1             0.799356             0.214474             0.010241   
2             0.967950             0.835396             0.533905   

                                                                  \
  Quantile_forecast_95 Quantile_forecast_96 Quantile_forecast_97   
0             0.400290             0.149113             0.826280   
1             0.507337             0.554264             0.283537   
2             0.786277             0.144454             0.909710   

                                             
  Quantile_forecast_98 Quantile_forecast_99  
0             0.198423             0.001391  
1             0.760090             0.423908  
2             0.477578             0.858335  

[3 rows x 990 columns]

In [149]:
exogeneous_space.sample(n_rows=3)

Site1                                  Site10                      \
        U10      U100       V10      V100       U10      U100       V10   
0 -0.735895 -1.001889 -0.040306  0.337505  1.245916  2.007525 -0.542370   
1  0.226214 -0.417589  0.445117 -0.477045  1.454729 -0.056192 -0.995153   
2 -0.920413  0.692832 -0.535392 -0.627698  0.070358  0.835419 -1.110435   

                Site2            ...     Site7               Site8            \
       V100       U10      U100  ...       V10      V100       U10      U100   
0 -0.262513  0.105425  0.032516  ... -0.384823  0.396762  0.401403 -0.772699   
1 -1.287023  0.551928 -0.478411  ...  0.485367  0.022178 -0.387284 -0.360958   
2  0.709685 -1.470986  1.205148  ...  0.652099  0.062170 -0.777496 -1.365857   

                          Site9                                
        V10      V100       U10      U100       V10      V100  
0 -1.194904  0.870400  0.658730 -0.039183  1.126278  0.836198  
1  0.274118  0.090201  0.845470 -0.582151 -0.305689  0.193123  
2 -0.787745 -1.003496  0.885643  1.205912 -0.824563  0.702019  

[3 rows x 40 columns]

In [150]:
action_space.sample(n_rows=3)

Site1                                            \
  Quantile_forecast_1 Quantile_forecast_10 Quantile_forecast_11   
0            0.923784             0.562542             0.837868   
1            0.304727             0.704436             0.080276   
2            0.224871             0.788183             0.271202   

                                                                  \
  Quantile_forecast_12 Quantile_forecast_13 Quantile_forecast_14   
0             0.459868             0.384162             0.456399   
1             0.218127             0.984153             0.859901   
2             0.622689             0.776736             0.752829   

                                                                  \
  Quantile_forecast_15 Quantile_forecast_16 Quantile_forecast_17   
0             0.081970             0.121961             0.214223   
1             0.666226             0.726099             0.960146   
2             0.077050             0.464065             0.082363   

                        ...                Site9                       \
  Quantile_forecast_18  ... Quantile_forecast_90 Quantile_forecast_91   
0             0.104984  ...             0.341300             0.415192   
1             0.088590  ...             0.461526             0.035954   
2             0.319574  ...             0.448183             0.053635   

                                                                  \
  Quantile_forecast_92 Quantile_forecast_93 Quantile_forecast_94   
0             0.872151             0.924470             0.970467   
1             0.708767             0.324796             0.548651   
2             0.032816             0.106432             0.902714   

                                                                  \
  Quantile_forecast_95 Quantile_forecast_96 Quantile_forecast_97   
0             0.533539             0.072975             0.948833   
1             0.339458             0.686280             0.412606   
2             0.454500             0.242988             0.432172   

                                             
  Quantile_forecast_98 Quantile_forecast_99  
0             0.829753             0.727098  
1             0.944068             0.009562  
2             0.341226             0.663856  

[3 rows x 990 columns]

### Step 3) Create environment

In [159]:
class GEFCom2014Wind(gym.Env):
    def __init__(self, dataset: ef.Dataset): 
        self.dataset = dataset
        self.data = dataset.data["data_gefcom2014_wind"]
        self.state_space = state_space
        self.exogeneous_space = exogeneous_space
        self.action_space = action_space
        self.idx_counter = 0

        self.train = [["2012-01-01 01:00:00", "2012-10-01 00:00:00"],
                      ["2012-10-01 01:00:00", "2012-11-01 00:00:00"],
                      ["2012-11-01 01:00:00", "2012-12-01 00:00:00"],
                      ["2012-12-01 01:00:00", "2013-01-01 00:00:00"],
                      ["2013-01-01 01:00:00", "2013-02-01 00:00:00"],
                      ["2013-02-01 01:00:00", "2013-03-01 00:00:00"],
                      ["2013-03-01 01:00:00", "2013-04-01 00:00:00"],
                      ["2013-04-01 01:00:00", "2013-05-01 00:00:00"],
                      ["2013-05-01 01:00:00", "2013-06-01 00:00:00"],
                      ["2013-06-01 01:00:00", "2013-07-01 00:00:00"],
                      ["2013-07-01 01:00:00", "2013-08-01 00:00:00"],
                      ["2013-08-01 01:00:00", "2013-09-01 00:00:00"],
                      ["2013-09-01 01:00:00", "2013-10-01 00:00:00"],
                      ["2013-10-01 01:00:00", "2013-11-01 00:00:00"],
                      ["2013-11-01 01:00:00", "2013-12-01 00:00:00"]]
        
        self.test = [["2012-10-01 01:00:00", "2012-11-01 00:00:00"],
                     ["2012-11-01 01:00:00", "2012-12-01 00:00:00"],
                     ["2012-12-01 01:00:00", "2013-01-01 00:00:00"],
                     ["2013-01-01 01:00:00", "2013-02-01 00:00:00"],
                     ["2013-02-01 01:00:00", "2013-03-01 00:00:00"],
                     ["2013-03-01 01:00:00", "2013-04-01 00:00:00"],
                     ["2013-04-01 01:00:00", "2013-05-01 00:00:00"],
                     ["2013-05-01 01:00:00", "2013-06-01 00:00:00"],
                     ["2013-06-01 01:00:00", "2013-07-01 00:00:00"],
                     ["2013-07-01 01:00:00", "2013-08-01 00:00:00"],
                     ["2013-08-01 01:00:00", "2013-09-01 00:00:00"],
                     ["2013-09-01 01:00:00", "2013-10-01 00:00:00"],
                     ["2013-10-01 01:00:00", "2013-11-01 00:00:00"],
                     ["2013-11-01 01:00:00", "2013-12-01 00:00:00"],
                     ["2013-12-01 01:00:00", "2014-01-01 00:00:00"]]
                
    def reset(self, return_dataframe=False):
        self.idx_counter = 0
        initial_dataframe = df.loc[(df.index.get_level_values('valid_datetime') >= self.train[0][0]) &
                                   (df.index.get_level_values('valid_datetime') <= self.train[0][1])]

        return initial_dataframe

    def step(self, action=None):
        
        next_state = df.loc[(df.index.get_level_values('valid_datetime') >= self.train[self.idx_counter+1][0]) &
                           (df.index.get_level_values('valid_datetime') <= self.train[self.idx_counter+1][1]),
                           pd.IndexSlice[:, 'Power']]

        next_exogenous = df.loc[(df.index.get_level_values('valid_datetime') >= self.test[self.idx_counter+1][0]) &
                    (df.index.get_level_values('valid_datetime') <= self.test[self.idx_counter+1][1]),
                    pd.IndexSlice[:, ['U10', 'V10', 'U100', 'V100']]] 

        done = True if self.idx_counter+2 == len(self.train) else False
        self.idx_counter += 1
        
        return next_state, next_exogenous, done

### Step 4) Create the model


In [161]:
from enflow.problems.objective import PinballLoss

### Step 5) Create the model


In [262]:
import lightgbm as lgb
import pandas as pd

class CatboostGEFCom2014(ef.Predictor):
    def __init__(self, quantiles=None):
        """
        Initialize the Predictor class.
        
        Args:
            quantiles (list): List of quantiles for which to create separate models.
                              Example: [0.1, 0.5, 0.9]
        """
        self.models = {}  # Dictionary to hold models for each site and quantile
        self.quantiles = [0.1, 0.5, 0.9] 
        
    def train(self, features: pd.DataFrame, target: pd.DataFrame, **kwargs):
        """
        Train separate LightGBM models for each site and quantile.
        
        Args:
            features (pd.DataFrame): Multi-indexed dataframe where the top-level index corresponds to sites.
            target (pd.DataFrame): The target dataframe (y), also multi-indexed by site.
            kwargs: Additional parameters to pass to the LightGBMRegressor model.
        """
        # Get the list of unique sites from the multi-index (top level)
        sites = features.columns.get_level_values(0).unique()
        feature_names = features.columns.get_level_values(1).unique()

        # Loop over each site
        for site in sites:
            # Extract the features and target for the current site
            site_features = features.xs(site, axis=1, level=0)
            site_target = target.xs(site, axis=1, level=0)

            # Loop over each quantile
            for quantile in self.quantiles:
                # Initialize a LightGBM model for this quantile
                params = {'objective': 'quantile', 'alpha': quantile, "verbose": -1}
                params.update(kwargs)  # Add any additional LightGBM parameters
                
                model = lgb.LGBMRegressor(**params)
                
                # Train the model on the site's data
                model.fit(site_features, site_target)
                
                # Store the trained model with a key (site, quantile)
                self.models[(site, quantile)] = model
                
    def predict(self, features: pd.DataFrame):
        """
        Make predictions for all sites and quantiles using the trained models.

        Args:
            features (pd.DataFrame): The feature dataframe (X), multi-indexed by site.

        Returns:
            pd.DataFrame: Multi-indexed DataFrame with predictions for each site and quantile.
        """
        # Get the list of unique sites from the multi-index (top level)
        sites = features.columns.get_level_values(0).unique()

        # Create a dictionary to store predictions
        predictions_dict = {}

        # Loop over each site and quantile
        for site in sites:
            # Extract the features for the current site
            site_features = features.xs(site, axis=1, level=0)

            for quantile in self.quantiles:
                # Check if the model for the given site and quantile exists
                if (site, quantile) not in self.models:
                    raise ValueError(f"No trained model for site '{site}' and quantile '{quantile}'.")

                # Make predictions using the stored model
                model = self.models[(site, quantile)]
                site_predictions = model.predict(site_features)

                # Store predictions in the dictionary with multi-index structure (site, quantile)
                predictions_dict[(site, quantile)] = site_predictions

        # Convert the predictions dictionary to a pandas DataFrame with multi-index columns
        predictions_df = pd.DataFrame(predictions_dict)

        # Set the multi-index columns (site, quantile) for the predictions DataFrame
        predictions_df.columns = pd.MultiIndex.from_tuples(predictions_df.columns, names=["Site", "Quantile"])

        return predictions_df


    def predict(self, features: pd.DataFrame):
        """
        Make predictions for a specific site and quantile using the trained model.
        
        Args:
            features (pd.DataFrame): The feature dataframe (X), multi-indexed by site.
            site (str): The site for which to make predictions.
            quantile (float): The quantile for which to make predictions.
        
        Returns:
            np.array: Predictions from the model.
        """

        # Create a nested dictionary to store predictions
        predictions = {}

        # Extract the features for the specific site
        sites = features.columns.get_level_values(0).unique()

        # Loop over each site and quantile
        for site in sites:
            # Extract the features for the current site
            site_features = features.xs(site, axis=1, level=0)

            # Initialize an inner dictionary for each site

            for quantile in self.quantiles:
                # Check if the model for the given site and quantile exists
                if (site, quantile) not in self.models:
                    raise ValueError(f"No trained model for site '{site}' and quantile '{quantile}'.")

                # Make predictions using the stored model
                model = self.models[(site, quantile)]
                site_predictions = model.predict(site_features)

                # Store the predictions under the quantile for the current site
                predictions[(site, f"quantile_{round(100*quantile)}")] = site_predictions

        # Convert the nested dictionary to a DataFrame with multi-index columns
        predictions = pd.DataFrame.from_dict(predictions)
        predictions.index = features.index
        
        return predictions


In [263]:
predictor = CatboostGEFCom2014()
predictor.train(features=df.loc[:,(slice(None), ["U10", "V10"])], 
                target=df.loc[:,(slice(None), "Power")])

In [264]:
predictor.predict(features=df.loc[:,(slice(None), ["U10", "V10"])])

Site1                          \
                                        quantile_10 quantile_50 quantile_90   
ref_datetime        valid_datetime                                            
2012-01-01 01:00:00 2012-01-01 01:00:00    0.019041    0.182683    0.450023   
                    2012-01-01 02:00:00    0.018177    0.183932    0.478619   
                    2012-01-01 03:00:00    0.038226    0.236469    0.520034   
                    2012-01-01 04:00:00    0.015072    0.156671    0.380125   
                    2012-01-01 05:00:00    0.011781    0.143335    0.423033   
...                                             ...         ...         ...   
2013-12-31 01:00:00 2013-12-31 20:00:00    0.129919    0.555189    0.862168   
                    2013-12-31 21:00:00    0.148537    0.595106    0.879154   
                    2013-12-31 22:00:00    0.261004    0.588266    0.939648   
                    2013-12-31 23:00:00    0.040923    0.188008    0.422063   
                    2014-01-01 00:00:00    0.040923    0.198971    0.593785   

                                              Site2                          \
                                        quantile_10 quantile_50 quantile_90   
ref_datetime        valid_datetime                                            
2012-01-01 01:00:00 2012-01-01 01:00:00    0.321280    0.698802    0.895029   
                    2012-01-01 02:00:00    0.256163    0.648614    0.843780   
                    2012-01-01 03:00:00    0.075746    0.452458    0.770182   
                    2012-01-01 04:00:00    0.033320    0.327028    0.641561   
                    2012-01-01 05:00:00    0.040495    0.239598    0.560727   
...                                             ...         ...         ...   
2013-12-31 01:00:00 2013-12-31 20:00:00    0.201635    0.756356    0.938464   
                    2013-12-31 21:00:00    0.406811    0.740943    0.941219   
                    2013-12-31 22:00:00    0.379019    0.630665    0.860346   
                    2013-12-31 23:00:00    0.224493    0.556199    0.871211   
                    2014-01-01 00:00:00    0.014966    0.102434    0.185507   

                                              Site3                          \
                                        quantile_10 quantile_50 quantile_90   
ref_datetime        valid_datetime                                            
2012-01-01 01:00:00 2012-01-01 01:00:00    0.316881    0.602427    0.857842   
                    2012-01-01 02:00:00    0.164322    0.364693    0.723257   
                    2012-01-01 03:00:00    0.082339    0.288610    0.609236   
                    2012-01-01 04:00:00    0.049286    0.258867    0.484978   
                    2012-01-01 05:00:00    0.016513    0.253836    0.481383   
...                                             ...         ...         ...   
2013-12-31 01:00:00 2013-12-31 20:00:00    0.089024    0.424835    0.756011   
                    2013-12-31 21:00:00    0.088832    0.521981    0.867599   
                    2013-12-31 22:00:00    0.171682    0.628052    0.894931   
                    2013-12-31 23:00:00    0.360569    0.704741    0.940998   
                    2014-01-01 00:00:00    0.212696    0.635272    0.943945   

                                              Site4  ...       Site7  \
                                        quantile_10  ... quantile_90   
ref_datetime        valid_datetime                   ...               
2012-01-01 01:00:00 2012-01-01 01:00:00    0.071743  ...    0.083236   
                    2012-01-01 02:00:00    0.005650  ...    0.099903   
                    2012-01-01 03:00:00    0.002001  ...    0.131530   
                    2012-01-01 04:00:00    0.001030  ...    0.246097   
                    2012-01-01 05:00:00    0.001397  ...    0.358210   
...                                             ...  ...         ...   
2013-12-31 01:00:00 2013-12-31 20:00:00    0.578162  ...    0.798065   
     

In [261]:
# Next steps: 
# 1) Run initial training and evaluate the scores
# 2) Implement the step function and calculate scores in a loop
# 3) Finally compare with the performance of the participants in the competition


### Step 6) Run the sequential decision loop and model evaluate performance

In [208]:
env = GEFCom2014Wind(dataset=dataset)
initial_df = env.reset()
initial_features = initial_df.loc[:,(slice(None), ["U10", "V10"])]
initial_target = initial_df.loc[:,(slice(None), "Power")]
predictor.train(features=initial_features, target=initial_target)

In [210]:
predictor

TypeError: 'CatboostGEFCom2014' object is not callable

In [137]:
state, exogenous, done = env.step()